In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import random
import cv2

torch.manual_seed(42)
# random.seed(42)

pd_submit = pd.read_csv('/kaggle/input/cat-dog-action-classification/sample_submit.csv')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import PIL

class CatDogDataset(Dataset):

    def __init__(self, root_dir, split, transform):
        """
        Args:
            root_dir (string): 이미지가 들어있는 'train_data' 와 'test_data'의 상위 폴더
            split (string): 'train' / 'test'
        """
        self.split = split
        self.root_dir = root_dir
        self.transform = transform
        
        if split == 'train':
            img_path = os.path.join(root_dir, 'train_data')
            self.img_paths = [ i for i in sorted(os.listdir(img_path))]
            self.labels = [i.split('_')[0] for i in sorted(os.listdir(img_path))]
        elif split == 'test':
            img_path = os.path.join(root_dir, 'test_data')
            self.img_paths = [i for i in sorted(os.listdir(img_path))]

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        if self.split == 'train':
            img_path = os.path.join(root_dir, 'train_data/' + self.img_paths[idx])
            img = PIL.Image.open(img_path)
            #img = cv2.imread(self.img_paths[idx])
            text_label_dict = {'arm':0, 'bodylower':1, 'sit':2, 'walkrun': 3, 'lying': 4}
            label = text_label_dict[self.labels[idx]]
                
            if self.transform:
                img = self.transform(img)
            return img, label
        elif self.split == 'test':
            img_path = os.path.join(root_dir, 'test_data/' + self.img_paths[idx])
            img = PIL.Image.open(img_path)
            #img = cv2.imread(self.img_paths[idx])
            if self.transform:
                img = self.transform(img)
            return img

In [ ]:
import torchvision

root_dir = '../input/cat-dog-action-classification'

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.RandomCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,244)),
    torchvision.transforms.ToTensor(),
])


train_dataset = CatDogDataset(root_dir = root_dir, split = 'train', transform = train_transform)
test_dataset = CatDogDataset(root_dir = root_dir, split = 'test', transform = test_transform)

train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=20,
                                          shuffle=True)

test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=20,
                                          shuffle=False)

total_batch = len(train_data_loader)

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import torchvision.models as models

# torch.manual_seed(42)
# random.seed(42)
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=5)
model = model.to(device)
# 기존 resnet50모델 대신 efficientnet을 사용함
# model = models.resnet50(pretrained = True)

In [ ]:
# model.fc

In [ ]:
# model.fc.out_featurse = 5


In [ ]:
# from efficientnet_pytorch import EfficientNet
# model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=5)
# model = model.to(device)

In [ ]:
nan

In [ ]:
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

# torch.manual_seed(42)
# random.seed(42)

model.train()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 5e-5)

epochs = 21 #0.41-> 베이스코드가 seed고정이 되어있지 않아서 0.47이 나옴
# epochs = 30
# epochs = 15

for epoch in tqdm(range(epochs)):
    avg_loss = 0
    for X, Y in train_data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        loss = loss_fn(hypothesis, Y)
        loss.backward()
        optimizer.step()
        
        avg_loss += loss / total_batch
        tmp_h = torch.Tensor()
        tmp_h = torch.cat([tmp_h,torch.argmax(hypothesis,1).cpu()])
        tmp_y = torch.Tensor()
        tmp_y = torch.cat([tmp_y, Y.cpu()])

    accuracy = sum(tmp_y == tmp_h) / len(tmp_y)
    print('epoch : {:4d}, Avg loss: {:.7f}, Accuracy: {:.7f}'.format(epoch , avg_loss, accuracy))


In [ ]:
nan

In [ ]:
with torch.no_grad():
    model.eval()
    pred_list = []
    for X_test in test_data_loader:
        prediction = model(X_test.to(device))
        argmax_pred = torch.argmax(prediction, dim = 1)
        pred_list.extend(argmax_pred.detach().cpu())

In [ ]:
pred_list = np.array(pred_list)

In [ ]:
pred_list

In [ ]:
pd_submit.head()

In [ ]:
pd_submit['Category'] = pred_list

In [ ]:
pd_submit.info()

In [ ]:
pd_submit = pd_submit.astype(str)

In [ ]:
pd_submit.info()

In [ ]:
pd_submit.to_csv('results.csv', index = False)